# Imports

In [1]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [2]:
url = "https://etfdailynews.com/etfs/Large-Cap-Blend-ETFs"

# Parsing TD cell with Line Break
Table cell is composed of two parts and presented on separate lines.

<img src="td_etf.png"> 

The td cell's HTML uses a br tag (line break) to split the presentation.

pandas.read_html is reading the text values and not processing the line break.

```html 
<td class="bold"><a class="show" href="/etf/SPY/">SPY<br/>
                    <span class="thirteen unbold">SPDR S&amp;P 500</span></a></td>
<!-- <td><a href="/etf/SPY/">SPDR S&P 500</a></td> -->
```
(HTML as of 3/18/18 https://etfdailynews.com/etfs/Large-Cap-Blend-ETFs/)


__Solution attempt 1:__ Find specific td with line breaks, add a semi-colon to br tag, split text on smi-colon.  Modify header for new fields.  Extract from Soup.

__not addressed:__ Datatype conversions.  All columns are objects



# Output from pandas read_html
Replay of NaT3z's observation.

In [3]:
%%time
sector_table = pd.read_html(url)


CPU times: user 636 ms, sys: 36.4 ms, total: 672 ms
Wall time: 3.6 s


In [4]:
sector_table[0].head(10)

,Fund Symbol/Name,Last,Open,High,Low,52-Wk Hi,52-Wk Lo,Day Chg,YTD Chg,12-Mo Chg,AUM,Exp. Ratio,SMART Grade
0,SPYSPDR S&P 500,274.20,274.50,275.39,274.14,286.63,231.61,-0.80(-0.29%),2.75%,14.98%,276.87B,0.09%,A Strong Buy
1,IVViShares Core S&P 500 ETF,277.30,277.66,278.53,277.30,288.69,233.10,+0.25(0.09%),3.14%,15.50%,158.06B,0.04%,A Strong Buy
2,VTIVanguard Total Stock Market ETF,141.66,141.58,142.19,141.58,146.87,118.89,+0.24(0.17%),3.21%,15.51%,95.99B,0.04%,A Strong Buy
3,VOOVanguard S&P 500 ETF,252.99,253.23,254.08,252.97,263.37,212.62,+0.26(0.10%),3.14%,15.52%,91.72B,0.04%,A Strong Buy
4,VIGVanguard Div Appreciation ETF - DNQ,103.93,103.80,104.27,103.71,108.99,88.73,+0.27(0.26%),1.86%,14.01%,29.60B,0.09%,A Strong Buy
5,IWBiShares Russell 1000 ETF,153.39,153.32,153.97,153.32,159.31,128.86,+0.26(0.17%),3.22%,15.57%,20.24B,0.15%,A Strong Buy
6,RSPGuggenheim S&P 500 Equal Weight ETF,102.44,102.15,102.75,102.15,107.44,88.88,-0.01(-0.01%),1.40%,12.03%,14.96B,0.20%,A Strong Buy
7,USMViShares Edge MSCI Min Vol USA ETF,53.26,53.12,53.37,53.12,55.45,47.44,+0.15(0.28%),0.91%,10.68%,14.57B,0.15%,A Strong Buy
8,ITOTiShares Core S&P Total U.S. Stock Market ETF,63.05,63.04,63.34,63.04,65.40,52.93,+0.06(0.10%),3.12%,15.48%,13.39B,0.03%,A Strong Buy
9,SCHXSchwab U.S. Large-Cap ETF,65.62,65.63,65.87,65.61,68.43,55.24,-0.21(-0.32%),2.88%,15.28%,12.30B,0.03%,A Strong Buy


In [5]:
sector_table[0]["Fund Symbol/Name"].iloc[0:10]

0                                     SPYSPDR S&P 500
1                         IVViShares Core S&P 500 ETF
2                  VTIVanguard Total Stock Market ETF
3                             VOOVanguard S&P 500 ETF
4              VIGVanguard Div Appreciation ETF - DNQ
5                         IWBiShares Russell 1000 ETF
6              RSPGuggenheim S&P 500 Equal Weight ETF
7               USMViShares Edge MSCI Min Vol USA ETF
8    ITOTiShares Core S&P Total U.S. Stock Market ETF
9                       SCHXSchwab U.S. Large-Cap ETF
Name: Fund Symbol/Name, dtype: object

In [6]:
# create a session

sesh = requests.session()

In [9]:
%%time
rsp = sesh.get(url)

CPU times: user 38.6 ms, sys: 7.69 ms, total: 46.2 ms
Wall time: 1.76 s


In [10]:

if rsp.ok:
    
    soup = BeautifulSoup(rsp.content, 'lxml')
    
    data_table = soup.findAll('table')[0]

In [11]:
def parse_etf_html(data_table, debug=False):
    '''
    First version of function to parse etf daily news table 
    
    Expected Input: Beautiful Soup table from https://etfdailynews.com/etfs/ * 
    
    Output: DataFrame
    
    '''
    
    header = [th.text for th in data_table.findAll('th')]

    # header modifications
    compound_field = header.pop(0)

    header.insert(0, compound_field.split('/')[0])
    header.insert(1, 'Fund ' + compound_field.split('/')[1])
    
    compound_field = header.pop(8)
    header.insert(8, compound_field + '_cur')
    header.insert(9, compound_field + '_per')
    
    
    # Row 0 is the table header
    
    extracted_data = list()
    
    # Starting at row 1, loop each table row 

    for tr in data_table.findAll('tr')[1:]:
        extracted_row = list()
        
        if debug:
            # simple test to verify if number of items matches expectations.
            row_parsing_log = dict()
    
        for td in tr.findAll('td'):
        
            #<td class="bold"><a class="show" href="/etf/SPY/">SPY<br/><span class="thirteen unbold">SPDR S&amp;P 500</span></a></td>,
            if td.find('a') and td.find('br') and td.find('span'):
                td.br.string=";"
                extracted_row.extend(td.text.split(";"))
                
                if debug:
                    row_parsing_log['symbol_fund_as_expected'] = len(td.text.split(";")) == 2
                
            # <td class="grade-4">+0.25<br/>(0.09%)</td>
            elif td.find('br') and [td.find('strong'),td.find('small'), td.find('a')]  == [None, None, None]:
                td.br.string=";"
                # percent change is enclosed with ().  remove to avoid confusion 
                extracted_row.append(td.text.split(";")[0])
                extracted_row.append(td.text.split(";")[1].replace("(", "").replace(")", ""))
                
                if debug:
                    row_parsing_log['day_chg_as_expected'] = len(td.text.split(";")) == 2
                
            #<td class="text-center grade-1"> <strong>A</strong><br/> <small>Strong Buy</small> </td>
            elif td.find('br') and td.find('strong') and td.find('small'):
                # Appears to be parsed correctly by pandas read html
                extracted_row.append(td.text.replace('\n', ' ').strip())
            
            else:
                extracted_row.append(td.text)
            
        
        record = dict(zip(header, extracted_row))
        
        if debug:
            record.update(row_parsing_log)
            
        # append each row
        extracted_data.append(record)


    if debug:
        header.extend(['symbol_fund_as_expected', 'day_chg_as_expected'])
        
    outputDF = pd.DataFrame(extracted_data)[header]
    
    # data types
    
    
    return outputDF
    
    

In [12]:
%%time
testDF = parse_etf_html(data_table)

CPU times: user 150 ms, sys: 0 ns, total: 150 ms
Wall time: 150 ms


In [14]:
testDF.head(10)

,Fund Symbol,Fund Name,Last,Open,High,Low,52-Wk Hi,52-Wk Lo,Day Chg_cur,Day Chg_per,YTD Chg,12-Mo Chg,AUM,Exp. Ratio,SMART Grade
0,SPY,SPDR S&P 500,274.20,274.50,275.39,274.14,286.63,231.61,-0.80,-0.29%,2.75%,14.98%,276.87B,0.09%,A Strong Buy
1,IVV,iShares Core S&P 500 ETF,277.30,277.66,278.53,277.30,288.69,233.10,+0.25,0.09%,3.14%,15.50%,158.06B,0.04%,A Strong Buy
2,VTI,Vanguard Total Stock Market ETF,141.66,141.58,142.19,141.58,146.87,118.89,+0.24,0.17%,3.21%,15.51%,95.99B,0.04%,A Strong Buy
3,VOO,Vanguard S&P 500 ETF,252.99,253.23,254.08,252.97,263.37,212.62,+0.26,0.10%,3.14%,15.52%,91.72B,0.04%,A Strong Buy
4,VIG,Vanguard Div Appreciation ETF - DNQ,103.93,103.80,104.27,103.71,108.99,88.73,+0.27,0.26%,1.86%,14.01%,29.60B,0.09%,A Strong Buy
5,IWB,iShares Russell 1000 ETF,153.39,153.32,153.97,153.32,159.31,128.86,+0.26,0.17%,3.22%,15.57%,20.24B,0.15%,A Strong Buy
6,RSP,Guggenheim S&P 500 Equal Weight ETF,102.44,102.15,102.75,102.15,107.44,88.88,-0.01,-0.01%,1.40%,12.03%,14.96B,0.20%,A Strong Buy
7,USMV,iShares Edge MSCI Min Vol USA ETF,53.26,53.12,53.37,53.12,55.45,47.44,+0.15,0.28%,0.91%,10.68%,14.57B,0.15%,A Strong Buy
8,ITOT,iShares Core S&P Total U.S. Stock Market ETF,63.05,63.04,63.34,63.04,65.40,52.93,+0.06,0.10%,3.12%,15.48%,13.39B,0.03%,A Strong Buy
9,SCHX,Schwab U.S. Large-Cap ETF,65.62,65.63,65.87,65.61,68.43,55.24,-0.21,-0.32%,2.88%,15.28%,12.30B,0.03%,A Strong Buy


In [15]:
testDF['Fund Symbol']

0       SPY
1       IVV
2       VTI
3       VOO
4       VIG
5       IWB
6       RSP
7      USMV
8      ITOT
9      SCHX
10       VV
11     SCHB
12      OEF
13     NOBL
14     DGRO
15     GSLC
16     MOAT
17      MGC
18     SPHQ
19      PKW
20      IYY
21     SPLG
22      JKD
23      DSI
24      CFO
25     LRGF
26     VONE
27      CSM
28      XLG
29      CFA
       ... 
110    VSDA
111    FLLV
112    VSMV
113     QVM
114    YLDE
115    PMOM
116    DIVB
117    CHGX
118     OEW
119    UDBI
120    USMF
121    SPDV
122    LOGO
123     OPD
124    SPXV
125    GMFL
126    SPMV
127     EEH
128    SECT
129    TMFC
130    GSEW
131     KLD
132    BERN
133     VSL
134    RVRS
135    OMFL
136    OSIZ
137    OYLD
138    OQAL
139    JBRI
Name: Fund Symbol, Length: 140, dtype: object